In [ ]:
%matplotlib inline
from __future__ import division;
from __future__ import print_function;
from __future__ import absolute_import;
import sys, os;
from collections import OrderedDict, namedtuple;
import numpy as np;

#import deepLIFT stuff
scriptsDir = os.environ.get("DEEPLIFT_DIR");
if (scriptsDir is None):
    raise Exception("Please set environment variable DEEPLIFT_DIR to point to the deeplift code (WITHIN the deeplift repo)");
sys.path.insert(0,scriptsDir);
#Make sure the directory is set to import the lab's version of keras
scriptsDir = os.environ.get("KERAS_DIR");
if (scriptsDir is None):
    raise Exception("Please set environment variable KERAS_DIR");
sys.path.insert(0,scriptsDir)

import keras_conversion as kc

In [ ]:
model_weights = "record_9_model_guiTJ_modelWeights.h5"
model_yaml = "record_9_model_guiTJ_modelYaml.yaml"
reload(kc)
keras_model = kc.load_keras_model(model_weights, model_yaml, normalise_conv_for_one_hot_encoded_input=True)

In [ ]:
scriptsDir = os.environ.get("UTIL_SCRIPTS_DIR");
if (scriptsDir is None):
    raise Exception("Please set environment variable UTIL_SCRIPTS_DIR to point to the deeplift code");
sys.path.insert(0,scriptsDir);
from importDataPackage import importData
reload(importData)
trainData, validData, testData = importData.loadTrainTestValidFromYaml("yaml/features.yaml","yaml/labels.yaml", "yaml/splits.yaml")

In [ ]:
data = testData

In [ ]:
import blobs
reload(blobs)
from blobs import MxtsMode
reload(kc)
import models
reload(models)

In [ ]:
deeplift_model = kc.convert_sequential_model(keras_model, mxts_mode=MxtsMode.DeepLIFT)
deeplift_contribs_func = deeplift_model.get_target_contribs_func(find_scores_layer_idx=0)
deeplift_multipliers_func = deeplift_model.get_target_multipliers_func(find_scores_layer_idx=0)

In [ ]:
gradients_model = kc.convert_sequential_model(keras_model, mxts_mode=MxtsMode.Gradient)
grad_times_inp_func = gradients_model.get_target_contribs_func(find_scores_layer_idx=0)
guided_backprop_model = kc.convert_sequential_model(keras_model, mxts_mode=MxtsMode.GuidedBackprop)
guided_backprop_func = guided_backprop_model.get_target_contribs_func(find_scores_layer_idx=0)
deconv_model = kc.convert_sequential_model(keras_model, mxts_mode=MxtsMode.DeconvNet)
deconv_func = deconv_model.get_target_contribs_func(find_scores_layer_idx=0)

In [ ]:
guided_backprop_deeplift1_model = kc.convert_sequential_model(keras_model, mxts_mode=MxtsMode.GuidedBackpropDeepLIFT1)
guided_backprop_deeplift1_func = guided_backprop_deeplift1_model.get_target_contribs_func(find_scores_layer_idx=0)

In [ ]:
guided_backprop_deeplift2_model = kc.convert_sequential_model(keras_model, mxts_mode=MxtsMode.GuidedBackpropDeepLIFT2)
guided_backprop_deeplift2_func = guided_backprop_deeplift2_model.get_target_contribs_func(find_scores_layer_idx=0)

In [ ]:
guided_backprop_deeplift3_model = kc.convert_sequential_model(keras_model, mxts_mode=MxtsMode.GuidedBackpropDeepLIFT3)
guided_backprop_deeplift3_func = guided_backprop_deeplift3_model.get_target_contribs_func(find_scores_layer_idx=0)

In [ ]:
expo_upweight_toplayers1_model = kc.convert_sequential_model(keras_model, mxts_mode=MxtsMode.ExpoUpweightTopLayers1)
expo_upweight_toplayers1_func = expo_upweight_toplayers1_model.get_target_contribs_func(find_scores_layer_idx=0)

In [ ]:
expo_upweight_toplayers2_model = kc.convert_sequential_model(keras_model, mxts_mode=MxtsMode.ExpoUpweightTopLayers2)
expo_upweight_toplayers2_func = expo_upweight_toplayers2_model.get_target_contribs_func(find_scores_layer_idx=0)

In [ ]:
guided_backprop_deeplift5_model = kc.convert_sequential_model(keras_model, mxts_mode=MxtsMode.GuidedBackpropDeepLIFT5)
guided_backprop_deeplift5_func = guided_backprop_deeplift5_model.get_target_contribs_func(find_scores_layer_idx=0)

In [ ]:
deeplift_contribs,\
deeplift_multipliers,\
grad_times_inp,\
guided_backprop,\
guided_backprop_deeplift1,\
guided_backprop_deeplift2,\
guided_backprop_deeplift3,\
expo_upweight_toplayers1,\
expo_upweight_toplayers2,\
guided_backprop_deeplift5,\
deconv = [np.array(contribs_func(task_idx=0, input_data_list=[data.X] , batch_size=10, progress_update=None))
                  for contribs_func in [deeplift_contribs_func,
                                        deeplift_multipliers_func,
                                        grad_times_inp_func,
                                        guided_backprop_func,
                                        guided_backprop_deeplift1_func,
                                        guided_backprop_deeplift2_func,
                                        guided_backprop_deeplift3_func,
                                        expo_upweight_toplayers1_func,
                                        expo_upweight_toplayers2_func,
                                        guided_backprop_deeplift5_func,
                                        deconv_func]]

In [ ]:
scriptsDir = os.environ.get("ENHANCER_SCRIPTS_DIR");
if (scriptsDir is None):
    raise Exception("Please set environment variable ENHANCER_SCRIPTS_DIR to point to the enhancer_prediction_code repo");
sys.path.insert(0,scriptsDir+"/featureSelector/deepLIFFT");
from deepLIFTutils import makePngOfSequenceDeepLIFTScores, plotWeights

In [ ]:
for (scores, scores_name) in [
            (grad_times_inp, 'grad_times_inp'),
            (guided_backprop, 'guided_backprop'),
            (deeplift_contribs, 'deeplift_contribs'),
            #(deeplift_multipliers, 'deeplift_multipliers'),                                  
            (guided_backprop_deeplift1, 'guided_backprop_deeplift1'),
            (guided_backprop_deeplift4, 'guided_backprop_deeplift4'),
            (guided_backprop_deeplift5, 'guided_backprop_deeplift5')]:
    print(scores_name)
    plotWeights(scores[4][0].T, bias=0, title="", figSize=(100,4), outputFile=None)

In [ ]:
deeplift_contribs.shape

In [ ]:
np.max(np.sqrt(np.abs(guided_backprop_deeplift4))*np.sign(guided_backprop_deeplift4))

In [ ]:
np.max(guided_backprop_deeplift4)

In [ ]:
np.max(deeplift_contribs)